# ANP Dataset

Kaggle dataset will be used to develop a visualization using gas stations from all over the country, their fuel prices each 'banner' was present.
Dataset extends throughout all the country, so the first sections on this notebook are about shrinking the dataset for optimization (time and memory) by choosing one region in particular. The development achieved in this dataset snippet will be used to the complete dataset.

## 1. Dataset adequation - snipping and shrinking
This dataset extends to different regions and states in Brazil.
To optimize time and computer resources, it would probably be a good idea to take just one of the regions the dataset refers to and sketch all the ETL that can happen in it. Hopefully the dataset is cohese enough for the other regions to follow the same path.

> Run this section only once to generate the reduced dataset!

In [ ]:
import pandas as pd

anp_data = pd.read_csv("anp.csv", sep=";", usecols=[0,1,2,3,4,10,11,12,13,15])
anp_data = anp_data.rename(columns={'Regiao - Sigla':'Regiao', 'Estado - Sigla':'Estado'})

# Filter for NE region
anp_data_NE = anp_data[anp_data["Regiao"] == "NE"]

# Writing file to be loaded for following sections
anp_data_NE.to_csv("northeast.csv", index=False)

## 2. ETL and data enrichment

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

# Northeast retailers only data load
# d_parser = lambda x: pd.to_datetime(x,'%Y-%m-%d')
anp_data_NE = pd.read_csv("../data/northeast.csv",
                           parse_dates=["Data da Coleta"]
                        #    parse_dates=["Data da Coleta"], date_parser=d_parser
                           )

In [6]:
# Dataset snip - Bahia only
is_Bahia = anp_data_NE["Estado"]=="BA"
isnt_GLP = anp_data_NE["Produto"]!="GLP"
anp_data_BA = anp_data_NE[is_Bahia & isnt_GLP]
anp_data_BA.head()

,Regiao,Estado,Municipio,Revenda,CNPJ da Revenda,Produto,Data da Coleta,Valor de Venda,Valor de Compra,Bandeira
1,NE,BA,JAGUAQUARA,LUZITALIA COM. E TRANSP. DE DERIVADOS DE PETRO...,01.073.545/0001-90,DIESEL,2004-05-10,1.229,1.153090,COSAN LUBRIFICANTES
2,NE,BA,SALVADOR,M DE AGUIAR COM. DE COMBUSTIVEIS E DER. DE PET...,34.310.961/0001-14,DIESEL,2004-05-10,1.200,1.155700,COSAN LUBRIFICANTES
3,NE,BA,SALVADOR,J A SOBRAL & CIA LTDA,13.799.101/0001-16,DIESEL,2004-05-10,1.239,1.178300,PETROBRAS DISTRIBUIDORA S.A.
4,NE,BA,SALVADOR,SA NACIONAL DE VEICULOS LTDA,15.226.699/0001-80,DIESEL,2004-05-10,1.300,1.696033,PETROBRAS DISTRIBUIDORA S.A.
5,NE,BA,SALVADOR,POSTOS MATARIPE ABASTECIMENTOS E SERVICOS LTDA,13.578.265/0001-13,DIESEL,2004-05-10,1.330,1.164200,PETROBRAS DISTRIBUIDORA S.A.


In [4]:
# City overall info - for table
city_overall = anp_data_BA
city_overall.loc[:, "Ano"] = city_overall["Data da Coleta"].dt.year
city_overall = city_overall.groupby(["Municipio","Ano","Produto"]).agg({"Revenda":"nunique", "Valor de Venda":["min","max"]}).reset_index()

new_column_names = []

for cols in city_overall.columns:
    new_col_name = f'{cols[0]}_{cols[1]}'
    new_column_names.append(new_col_name)
city_overall.columns = new_column_names
city_overall.columns = ['Municipio', 'Ano', 'Produto', 'Numero de Revendas', 'Valor de Venda (min)', 'Valor de Venda (max)']


# Daily average for all gas stations, by fuel type
daily_fuel_avg = anp_data_BA.groupby(["Produto","Data da Coleta"])["Valor de Venda"].agg(["mean"]).reset_index()
daily_fuel_avg.columns = ["Produto", "Data da Coleta", "Valor de Venda medio"]
daily_fuel_avg.loc[:, "Ano"] = daily_fuel_avg["Data da Coleta"].dt.year

# Calculates average for every fuel sold in every city
city_alltime_avg = anp_data_BA.groupby(["Municipio","Produto"])["Valor de Venda"].agg("mean")
city_alltime_avg = city_alltime_avg.reset_index()

# Calculates maximum of the sum of the average fuel prices for each city.
# Then normalizes all averages calculated on "city_alltime_avg" by this maximum value.
city_alltime_avg_sum = city_alltime_avg.groupby("Municipio")["Valor de Venda"].agg("sum").reset_index()
divisor = city_alltime_avg_sum["Valor de Venda"].max()
city_alltime_avg["Normalized"] = city_alltime_avg["Valor de Venda"].apply(lambda x:x/divisor)

# Ordering by the city with the highest overall means
city_order = city_alltime_avg_sum.sort_values("Valor de Venda", ascending=False)
city_ordered_list = city_order.set_index("Municipio").index

city_alltime_avg["Municipio"]= pd.Categorical(city_alltime_avg["Municipio"], categories=city_ordered_list)


In [5]:
from dash import Dash, html, dcc, dash_table, callback, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px

app = Dash(__name__, external_stylesheets=[dbc.themes.SLATE])
app.layout = html.Div([
    dcc.Markdown("## ANP retail data "),
    html.Br(),
    dbc.Row([
        dbc.Col([
            html.H4("Select a range in years"),
            dcc.RangeSlider(
                id = "all_time_slider",
                min = daily_fuel_avg["Data da Coleta"].dt.year.unique().tolist()[0],
                max = daily_fuel_avg["Data da Coleta"].dt.year.unique().tolist()[-1],
                value = [
                        daily_fuel_avg["Data da Coleta"].dt.year.unique().tolist()[-5],
                        daily_fuel_avg["Data da Coleta"].dt.year.unique().tolist()[-1]
                        ],
                step = 1,
                marks = {i: str(i) for i in range(
                            daily_fuel_avg["Data da Coleta"].dt.year.unique().tolist()[0],
                            daily_fuel_avg["Data da Coleta"].dt.year.unique().tolist()[-1],
                            1)
                        },
            ),
            dcc.Graph(
                id = "fuel_avg",
            ),
        ]),

        dbc.Col([
            html.H4("Select a range in years"),
            dcc.RangeSlider(
                id = "slider",
                min = daily_fuel_avg["Data da Coleta"].dt.year.unique().tolist()[0],
                max = daily_fuel_avg["Data da Coleta"].dt.year.unique().tolist()[-1],
                value = [
                        daily_fuel_avg["Data da Coleta"].dt.year.unique().tolist()[-5],
                        daily_fuel_avg["Data da Coleta"].dt.year.unique().tolist()[-1]
                        ],
                step = 1,
                marks = {i: str(i) for i in range(
                            daily_fuel_avg["Data da Coleta"].dt.year.unique().tolist()[0],
                            daily_fuel_avg["Data da Coleta"].dt.year.unique().tolist()[-1],
                            1)
                        },
            ),
            dash_table.DataTable(
                id = "tbl_out",
                data = city_overall.to_dict('records'),
                columns=[{"name": i, "id": i} for i in city_overall.columns],
                sort_action="native",
                page_size=10,
                merge_duplicate_headers=True
            )
        ])
    ]),

    dbc.Row([
        dbc.Col([
            dcc.Graph(
                id = "city_alltime_avg",
                figure = px.bar(
                    city_alltime_avg.reset_index().sort_values("Municipio",ascending=True),
                    title = "City all-time average, all products",
                    x = "Municipio",
                    y = "Normalized",
                    color = "Produto",
                )
            ),
        ])


    ])
])

# Line chart callback

@callback(Output("fuel_avg", "figure"),
          Input("all_time_slider", "value")
         )
def update_all_time(slider_value):
    figure = px.line(
        daily_fuel_avg.reset_index()[daily_fuel_avg.Ano.between(slider_value[0], slider_value[1])],
        title = "Daily avg by fuel - all gas stations",
        x = "Data da Coleta",
        y = "Valor de Venda medio",
        color = "Produto"
    )
    return figure

# Table callback

@callback(Output("tbl_out", "data"),
          Input("slider", "value")
         )
def update_table(slider_value):
    if not slider_value:
        return dash.no_update
    dff = city_overall[city_overall.Ano.between(slider_value[0], slider_value[1])]
    return dff.to_dict("records")

if __name__ == "__main__":
  app.run(jupyter_mode="external")

Dash app running on http://127.0.0.1:8050/


In [ ]:
# Revenda field investigation:
revenda_estimated_duration = anp_data_BA.groupby(["CNPJ da Revenda", "Bandeira"])["Data da Coleta"].agg(["min","max"]).reset_index()
revenda_estimated_duration.columns = ["CNPJ da Revenda","Bandeira","Primeira Data","Ultima Data"]
revenda_estimated_duration = revenda_estimated_duration.sort_values(["CNPJ da Revenda","Primeira Data"], ascending=[True,True])

# Bandeiras count throughout retail life
count_revenda = anp_data_BA.groupby("CNPJ da Revenda")["Bandeira"].nunique().reset_index()
count_revenda.columns = ["CNPJ da Revenda","Bandeiras"]
revenda_estimated_duration.head()

def today_diff(rows):
    last_date = rows.loc["Ultima Data"]
    today = pd.Timestamp.today().date()
    diff = (pd.to_datetime(today) - pd.to_datetime(last_date)).days
    if diff >= 30:
        return "Old"
    else:
        return "Keep"

# np.sort(anp_data_BA["Municipio"].unique())
# estimated_duration["Diff"] = estimated_duration.apply(today_diff, axis="columns")
revenda_estimated_duration["Diff"] = revenda_estimated_duration.apply(today_diff, axis="columns")
revenda_estimated_duration[revenda_estimated_duration["Diff"] == "Keep"]